In [1]:
import yt
import yt.units as u
import pysac.yt
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
l = 0
#r0 = 0.09 * u.Mm
r0 = 0.1 * u.Mm
period = 120
global t
t = 0
amp = 1.e4
mu0 = 4.e-7 * np.pi * u.N / u.A**2


def tdep():
    if t < period / 2.0:
        return np.sin(t* 2.0 * np.pi / period)
    else:
        return 0.0


def _alfvenspeed(field, data):
    bx, by, bz = data['mag_field_x_bg'], data['mag_field_y_bg'], data['mag_field_z_bg']
    B = np.sqrt((bx**2 + by**2 + bz**2) / mu0)
    return u.yt_array.YTArray(B / np.sqrt(data['density_bg']), B.units / np.sqrt(data['density_bg']).units)


def _theta(field, data):
    x, y = data['x'], data['y']
    return u.yt_array.YTArray(np.arctan2(y, x), 'radian')


def _r(field, data):
    x, y = data['x'], data['y']
    return u.yt_array.YTArray(np.sqrt(x**2 + y**2), 'm')


def _v_theta(field, data):
    v_A, r, theta = data['alfvenspeed'], data['r'] / r0, data['theta']
    #return u.yt_array.YTArray(v_A * r * (1 - r**2) * np.exp(-(r**2)) * np.cos(l * theta) * tdep(), 'Mm/s')
    return u.yt_array.YTArray(amp * r * (1 - r**2) * np.exp(-(r**2)) * np.cos(l * theta) * tdep(), 'm/s')


def _v_r(field, data):
    v_A, r, theta = data['alfvenspeed'], data['r'] / r0, data['theta']
    #return u.yt_array.YTArray(v_A * l * (r / 2.0) * np.exp(-(r**2)) * np.sin(l * theta) * tdep(), 'Mm/s')
    return u.yt_array.YTArray(amp * l * (r / 2.0) * np.exp(-(r**2)) * np.sin(l * theta) * tdep(), 'm/s')


def _v_x(field, data):
    u_r, u_th, theta = data['v_r'], data['v_theta'], data['theta']
    return u.yt_array.YTArray((u_r * np.cos(theta)) - (u_th * np.sin(theta)), 'm/s')


def _v_y(field, data):
    u_r, u_th, theta = data['v_r'], data['v_theta'], data['theta']
    return u.yt_array.YTArray((u_r * np.sin(theta)) + (u_th * np.cos(theta)), 'm/s')


yt.add_field('alfvenspeed', function=_alfvenspeed, units='km/s')
yt.add_field('theta', function=_theta, units='radian')
yt.add_field('r', function=_r, units='m')
yt.add_field('v_theta', function=_v_theta, units='km/s')
yt.add_field('v_r', function=_v_r, units='km/s')
yt.add_field('v_x', function=_v_x, units='km/s')
yt.add_field('v_y', function=_v_y, units='km/s')

In [3]:
#thisfile = yt.load('/data/sm1ajl/mhs_atmosphere/drew_model/drew_model.gdf')
#thisfile = yt.load('/fastdata/sm1ajl/Flux-Surfaces/gdf/m0_p120-0_0-5_0-5/*00001.gdf')
#thisfile = yt.load('/data/sm1ajl/mhs_atmosphere/drew_paper1/drew_paper1.gdf')
thisfile = yt.load('/data/sm1ajl/mhs_atmosphere/mfe_setup/mfe_setup.gdf')

yt : [WARNING  ] 2016-06-07 16:47:06,802 'field_units' was overridden by 'dataset_units/density_bg'
yt : [WARNING  ] 2016-06-07 16:47:06,804 'field_units' was overridden by 'dataset_units/density_pert'
yt : [WARNING  ] 2016-06-07 16:47:06,806 'field_units' was overridden by 'dataset_units/internal_energy_bg'
yt : [WARNING  ] 2016-06-07 16:47:06,807 'field_units' was overridden by 'dataset_units/internal_energy_pert'
yt : [WARNING  ] 2016-06-07 16:47:06,809 'field_units' was overridden by 'dataset_units/mag_field_x_bg'
yt : [WARNING  ] 2016-06-07 16:47:06,811 'field_units' was overridden by 'dataset_units/mag_field_x_pert'
yt : [WARNING  ] 2016-06-07 16:47:06,812 'field_units' was overridden by 'dataset_units/mag_field_y_bg'
yt : [WARNING  ] 2016-06-07 16:47:06,813 'field_units' was overridden by 'dataset_units/mag_field_y_pert'
yt : [WARNING  ] 2016-06-07 16:47:06,815 'field_units' was overridden by 'dataset_units/mag_field_z_bg'
yt : [WARNING  ] 2016-06-07 16:47:06,816 'field_units' w

In [4]:
print thisfile.parameters['gravity0']
print thisfile.parameters['gravity1']
print thisfile.parameters['gravity2']
print thisfile.parameters['gravity3']

print thisfile.index.grids[0]['density_bg'].min(), thisfile.index.grids[0]['density_bg'].max()
total = thisfile.index.grids[0]['density_bg'] + thisfile.index.grids[0]['density_pert']
print total.min(), total.max()
print thisfile.index.grids[0]['alfvenspeed'].min(), thisfile.index.grids[0]['alfvenspeed'].max()
print thisfile.index.grids[0]['v_theta'].min(), thisfile.index.grids[0]['v_theta'].max()
#t = 30

0.0
0.0
-274.0


KeyError: 'gravity3'

In [ ]:
myplot = yt.SlicePlot(thisfile, 'x', 'magnetic_field_strength', axes_unit='Mm', origin='native')
myplot.set_cmap('magnetic_field_strength', 'viridis')
#myplot.set_cmap('plasma_beta', 'magma')
#myplot.annotate_streamlines('mag_field_y', 'mag_field_z', plot_args={'color': 'grey'})

seed_points = np.zeros([11,2]) + 1.52
seed_points[:,0] = np.linspace(-0.99, 0.95, seed_points.shape[0],
                               endpoint=True)

#min, max = thisfile.all_data().quantities.extrema("magnetic_field_strength")
#norm = mpl.colors.LogNorm(min.value+1e-5, max.value)
#myplot.annotate_streamlines('mag_field_y', 'mag_field_z',
myplot.annotate_streamlines('mag_field_x', 'mag_field_y',
                         #field_color='magnetic_field_strength',
                         plot_args={'start_points': seed_points,
                                    'density': 15,
                                    'cmap': 'plasma', 'linewidth':2,
                                    #'norm':norm,
                                    'color': 'grey'
                                    })
myplot.save('/fastdata/sm1ajl/Flux-Surfaces/figs/driverplots/m{}/'.format(l))
myplot.show()

In [ ]:
myplot = yt.SlicePlot(thisfile, 'x', 'magnetic_field_strength', axes_unit='Mm', origin='native')
myplot.set_cmap('magnetic_field_strength', 'plasma')
myplot.annotate_streamlines('mag_field_y', 'mag_field_z',
                         #field_color='magnetic_field_strength',
                         plot_args={'start_points': seed_points,
                                    'density': 15,
                                    'cmap': 'plasma', 'linewidth':2,
                                    'norm':norm,
                                    'color': 'grey'
                                    })
myplot.save('/fastdata/sm1ajl/Flux-Surfaces/figs/driverplots/m{}/'.format(l))
myplot.show()

In [ ]:
myplot = yt.SlicePlot(thisfile, 'x', 'alfvenspeed', axes_unit='Mm', origin='native')
myplot.set_cmap('alfvenspeed', 'viridis')
myplot.annotate_streamlines('mag_field_y', 'mag_field_z',
                         #field_color='magnetic_field_strength',
                         plot_args={'start_points': seed_points,
                                    'density': 15,
                                    'cmap': 'plasma', 'linewidth':2,
                                    'norm':norm,
                                    'color': 'grey'
                                    })
myplot.save('/fastdata/sm1ajl/Flux-Surfaces/figs/driverplots/m{}/'.format(l))
myplot.show()

In [ ]:
myplot = yt.SlicePlot(thisfile, 'z', 'alfvenspeed', axes_unit='Mm',
                     center=[0.0, 0.0, 0.0]*u.Mm)
myplot.set_cmap('alfvenspeed', 'viridis')
myplot.annotate_quiver('v_x', 'v_y', scale=0.1)
myplot.zoom(4)
myplot.save('/fastdata/sm1ajl/Flux-Surfaces/figs/driverplots/m{}/'.format(l))
myplot.show()

In [ ]:
for t in np.arange(0.0, 70.0, 10.0):#2.5):
    #myplot = yt.SlicePlot(thisfile, 'z', 'mag_field_z_bg', axes_unit='Mm',
    myplot = yt.SlicePlot(thisfile, 'z', 'alfvenspeed', axes_unit='Mm',
                         center=[0.0, 0.0, 0.1])
    #myplot.set_cmap('mag_field_z_bg', 'coolwarm')
    myplot.set_cmap('alfvenspeed', 'viridis')
    myplot.annotate_quiver('v_x', 'v_y', scale=0.05)
    #myplot.zoom(4)
    myplot.show()
    #myplot.save('/fastdata/sm1ajl/Flux-Surfaces/figs/driverplots/m{}/{:04}'.format(l, t))
    print 'figs/driverplots/m{}/{:04}'.format(l, t), thisfile.all_data().quantities.extrema("v_theta")

In [ ]:
#B_sq = (Bx_bg + Bx_p)**2 + (By_bg + By_p)**2 + (Bz_bg + Bx_p)**2
#v_A = SQRT(B_sq / (rho_bg + rho_p))
#l = 0
for m in range(4):
    print 'm = ', m
    v_A = 1.0
    alpha = 1.0
    tdep = 1.0

    xx = np.linspace(-1.0e6, 1.0e6, 256)
    yy = np.linspace(-1.0e6, 1.0e6, 256)
    zz = np.linspace(0, 1.6e6, 128)

    #x, y, z = np.meshgrid(xx, yy, zz)
    x, y = np.meshgrid(xx, yy)

    r0 = 0.5e6
    r = np.sqrt(x**2 + y**2) / r0
    theta = np.arctan2(y, x)

    exp_z = 1.0 #np.exp(-(z**2)/(0.05e6**2))
    fig, ax = plt.subplots(1, 2, figsize=(24, 12))
    fig.patch.set_facecolor('white')

    for i, l in enumerate([-m, m]):
        print 'm = ', l
        u_th = v_A * alpha * r * (1 - r**2) * np.exp(-(r**2)) * np.cos(l * theta)# * tdep * exp_z
        u_r = v_A * alpha * l * (r / 2.0) * np.exp(-(r**2)) * np.sin(l * theta)# * tdep * exp_z

        ux = ((u_r * np.cos(theta)) - (u_th * np.sin(theta)))[::4, ::4]
        uy = ((u_r * np.sin(theta)) + (u_th * np.cos(theta)))[::4, ::4]

        u = np.sqrt(ux**2 + uy**2)

        ax[i].imshow(u, cmap='gist_gray', origin='lower', extent=[xx[0], xx[-1], yy[0], yy[-1]])
        ax[i].quiver(x[::4, ::4], y[::4, ::4], ux, uy, scale=8.0, color='gray')
        ax[i].set_title('m = {}'.format(l))
    plt.savefig('/fastdata/sm1ajl/Flux-Surfaces/drivershape_m={}'.format(m),
               facecolor='white', transparent=False)
    plt.close()

In [ ]:
#B_sq = (Bx_bg + Bx_p)**2 + (By_bg + By_p)**2 + (Bz_bg + Bx_p)**2
#v_A = SQRT(B_sq / (rho_bg + rho_p))
#l = 0

v_A = 1.0
alpha = 1.0
tdep = 1.0

xx = np.linspace(-1.0e6, 1.0e6, 256)
yy = np.linspace(-1.0e6, 1.0e6, 256)
zz = np.linspace(0, 1.6e6, 128)

x, y = np.meshgrid(xx, yy)
ux = np.zeros(x.shape)
uy = np.zeros(x.shape)

r0 = 0.5e6
r = np.sqrt(x**2 + y**2) / r0
theta = np.arctan2(y, x)

exp_z = 1.0 #np.exp(-(z**2)/(0.05e6**2))


for l in range(4):
    print 'm = ', l

    fig = plt.figure(figsize=(14, 12))

    u_th = v_A * alpha * r * (1 - r**2) * np.exp(-(r**2)) * np.cos(l * theta)# * tdep * exp_z
    u_r = v_A * alpha * l * (r / 2.0) * np.exp(-(r**2)) * np.sin(l * theta)# * tdep * exp_z

    ux += ((u_r * np.cos(theta)) - (u_th * np.sin(theta)))#[::4, ::4]
    uy += ((u_r * np.sin(theta)) + (u_th * np.cos(theta)))#[::4, ::4]

    u = np.sqrt(ux**2 + uy**2)

    plt.imshow(u, cmap='gist_gray', origin='lower', extent=[xx[0], xx[-1], yy[0], yy[-1]])
    plt.quiver(x[::4, ::4], y[::4, ::4], ux[::4, ::4], uy[::4, ::4], scale=10.0, color='gray')
    plt.savefig('/fastdata/sm1ajl/Flux-Surfaces/sumdrivershape_m={}'.format(l))
    plt.close()

In [ ]:
fig = plt.figure()
plt.plot(r[:, 63], u_th[:, 63] / (v_A * alpha))
plt.xlabel('$r/r_0$')
plt.ylabel('$v_{th}/a v_A$')
plt.show()